##### Copyright 2022 The TensorFlow Compression Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 학습된 데이터 압축

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/tutorials/generative/data_compression"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/tutorials/generative/data_compression.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행하기</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/tutorials/generative/data_compression.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/tutorials/generative/data_compression.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드하기</a></td>
</table>

## 개요

이 노트북은 신경망과 [TensorFlow Compression](https://github.com/tensorflow/compression)을 사용하여 손실 데이터 압축을 수행하는 방법을 보여줍니다.

손실 압축 시에는 샘플을 인코딩하는 데 필요한 예상 비트 수인 **rate**와 샘플 재구성에서 예상되는 오류인 **왜곡** 사이의 절충이 포함됩니다.

아래 예에서는 자동 인코더와 유사한 모델을 사용하여 MNIST 데이터세트의 이미지를 압축합니다. 이 방법은 [End-to-end Optimized Image Compression](https://arxiv.org/abs/1611.01704) 논문을 기초로 합니다.

학습된 데이터 압축에 대한 더 많은 배경 지식은 기존 데이터 압축에 익숙한 사람들을 대상으로 하는 [이 백서](https://arxiv.org/abs/2007.03034) 또는 머신 러닝에 관련된 사람들을 대상으로 하는 [이 설문조사](https://arxiv.org/abs/2202.06533)에서 찾을 수 있습니다.


## 설정

`pip`를 통해 Tensorflow Compression을 설치합니다.

In [ ]:
%%bash
# Installs the latest version of TFC compatible with the installed TF version.

read MAJOR MINOR <<< "$(pip show tensorflow | perl -p -0777 -e 's/.*Version: (\d+)\.(\d+).*/\1 \2/sg')"
pip install "tensorflow-compression<$MAJOR.$(($MINOR+1))"


라이브러리 종속성을 가져옵니다.

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_compression as tfc
import tensorflow_datasets as tfds


## 트레이너 모델 정의하기

모델이 자동 인코더와 유사하고 훈련 및 추론 중에 다른 기능들을 수행해야 하기 때문에 설정은 예를 들어 분류자와 약간 다릅니다.

훈련 모델은 세 부분으로 구성됩니다.

- **분석**(또는 인코더) 변환: 이미지에서 잠재 공간으로 변환
- **합성**(또는 디코더) 변환: 잠재 공간에서 이미지 공간으로 다시 변환
- **Prior** 및 엔트로피 모델: 잠재 공간의 한계 확률 모델링

먼저 변환을 정의합니다.

In [ ]:
def make_analysis_transform(latent_dims):
  """Creates the analysis (encoder) transform."""
  return tf.keras.Sequential([
      tf.keras.layers.Conv2D(
          20, 5, use_bias=True, strides=2, padding="same",
          activation="leaky_relu", name="conv_1"),
      tf.keras.layers.Conv2D(
          50, 5, use_bias=True, strides=2, padding="same",
          activation="leaky_relu", name="conv_2"),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(
          500, use_bias=True, activation="leaky_relu", name="fc_1"),
      tf.keras.layers.Dense(
          latent_dims, use_bias=True, activation=None, name="fc_2"),
  ], name="analysis_transform")


In [ ]:
def make_synthesis_transform():
  """Creates the synthesis (decoder) transform."""
  return tf.keras.Sequential([
      tf.keras.layers.Dense(
          500, use_bias=True, activation="leaky_relu", name="fc_1"),
      tf.keras.layers.Dense(
          2450, use_bias=True, activation="leaky_relu", name="fc_2"),
      tf.keras.layers.Reshape((7, 7, 50)),
      tf.keras.layers.Conv2DTranspose(
          20, 5, use_bias=True, strides=2, padding="same",
          activation="leaky_relu", name="conv_1"),
      tf.keras.layers.Conv2DTranspose(
          1, 5, use_bias=True, strides=2, padding="same",
          activation="leaky_relu", name="conv_2"),
  ], name="synthesis_transform")


트레이너는 두 변환의 인스턴스와 이전의 매개변수를 모두 보유합니다.

`call` 메서드는 다음을 계산하도록 설정됩니다.

- **비율**: 숫자 배치를 나타내는 데 필요한 비트 수에 대한 추정치
- **왜곡**: 원래 숫자의 픽셀과 재구성된 픽셀 간의 평균 절대 차이


In [ ]:
class MNISTCompressionTrainer(tf.keras.Model):
  """Model that trains a compressor/decompressor for MNIST."""

  def __init__(self, latent_dims):
    super().__init__()
    self.analysis_transform = make_analysis_transform(latent_dims)
    self.synthesis_transform = make_synthesis_transform()
    self.prior_log_scales = tf.Variable(tf.zeros((latent_dims,)))

  @property
  def prior(self):
    return tfc.NoisyLogistic(loc=0., scale=tf.exp(self.prior_log_scales))

  def call(self, x, training):
    """Computes rate and distortion losses."""
    # Ensure inputs are floats in the range (0, 1).
    x = tf.cast(x, self.compute_dtype) / 255.
    x = tf.reshape(x, (-1, 28, 28, 1))

    # Compute latent space representation y, perturb it and model its entropy,
    # then compute the reconstructed pixel-level representation x_hat.
    y = self.analysis_transform(x)
    entropy_model = tfc.ContinuousBatchedEntropyModel(
        self.prior, coding_rank=1, compression=False)
    y_tilde, rate = entropy_model(y, training=training)
    x_tilde = self.synthesis_transform(y_tilde)

    # Average number of bits per MNIST digit.
    rate = tf.reduce_mean(rate)

    # Mean absolute difference across pixels.
    distortion = tf.reduce_mean(abs(x - x_tilde))

    return dict(rate=rate, distortion=distortion)


### 비율과 왜곡 계산하기

훈련 세트의 이미지 하나를 사용하여 이를 단계별로 살펴보겠습니다. 훈련 및 검증을 위해 MNIST 데이터세트를 로드합니다.

In [ ]:
training_dataset, validation_dataset = tfds.load(
    "mnist",
    split=["train", "test"],
    shuffle_files=True,
    as_supervised=True,
    with_info=False,
)


그리고 하나의 이미지 $x$를 추출합니다.

In [ ]:
(x, _), = validation_dataset.take(1)

plt.imshow(tf.squeeze(x))
print(f"Data type: {x.dtype}")
print(f"Shape: {x.shape}")


잠재 표현 $y$를 얻으려면 `float32`로 캐스팅하고 배치 차원을 추가하고 분석 변환을 통해 전달해야 합니다.

In [ ]:
x = tf.cast(x, tf.float32) / 255.
x = tf.reshape(x, (-1, 28, 28, 1))
y = make_analysis_transform(10)(x)

print("y:", y)


잠재 공간은 테스트 시간에 양자화됩니다. 훈련 중에 이를 미분 가능한 방식으로 모델링하기 위해 $(-.5, .5)$ 구간에 균일한 노이즈를 추가하고 결과를 $\tilde y$라고 합니다. 이 용어는 [End-to-end Optimized Image Compression](https://arxiv.org/abs/1611.01704) 논문에서 사용된 것과 동일합니다.

In [ ]:
y_tilde = y + tf.random.uniform(y.shape, -.5, .5)

print("y_tilde:", y_tilde)


"Prior"는 노이즈가 많은 잠재 공간의 한계 분포를 모델링하기 위해 훈련하는 확률 밀도입니다. 예를 들어, 각 잠재 차원에 대해 다른 척도를 가진 독립적인 [로지스틱 분포](https://en.wikipedia.org/wiki/Logistic_distribution) 세트일 수 있습니다. `tfc.NoisyLogistic`은 잠재 공간에 부가적인 노이즈가 있다는 사실을 설명합니다. 척도가 0에 가까워지면 로지스틱 분포가 dirac 델타(스파이크)에 접근하지만 추가된 노이즈로 인해 "노이즈가 많은" 분포가 대신 균일 분포에 접근합니다.

In [ ]:
prior = tfc.NoisyLogistic(loc=0., scale=tf.linspace(.01, 2., 10))

_ = tf.linspace(-6., 6., 501)[:, None]
plt.plot(_, prior.prob(_));


훈련하는 동안 `tfc.ContinuousBatchedEntropyModel`은 균일한 노이즈를 추가하고 노이즈와 prior를 사용하여 비율(잠재 표현을 인코딩하는 데 필요한 평균 비트 수)에 대한 (미분 가능한) 상한을 계산합니다. 그 경계는 손실로 최소화할 수 있습니다.

In [ ]:
entropy_model = tfc.ContinuousBatchedEntropyModel(
    prior, coding_rank=1, compression=False)
y_tilde, rate = entropy_model(y, training=True)

print("rate:", rate)
print("y_tilde:", y_tilde)


마지막으로, 노이즈가 많은 잠재 공간은 합성 변환을 통해 다시 전달되어 이미지 재구성 $\tilde x$를 생성합니다. 왜곡은 원본 이미지와 재구성 사이의 오차입니다. 훈련되지 않은 변환을 사용하면 재구성이 그다지 유용하지 않다는 것은 분명합니다.

In [ ]:
x_tilde = make_synthesis_transform()(y_tilde)

# Mean absolute difference across pixels.
distortion = tf.reduce_mean(abs(x - x_tilde))
print("distortion:", distortion)

x_tilde = tf.saturate_cast(x_tilde[0] * 255, tf.uint8)
plt.imshow(tf.squeeze(x_tilde))
print(f"Data type: {x_tilde.dtype}")
print(f"Shape: {x_tilde.shape}")


모든 숫자 배치에 대해 `MNISTCompressionTrainer`를 호출하면 해당 배치에 대한 평균으로 비율과 왜곡이 생성됩니다.


In [ ]:
(example_batch, _), = validation_dataset.batch(32).take(1)
trainer = MNISTCompressionTrainer(10)
example_output = trainer(example_batch)

print("rate: ", example_output["rate"])
print("distortion: ", example_output["distortion"])


다음 섹션에서는 이러한 두 손실에 대해 경사하강법을 수행하도록 모델을 설정합니다.

## 모델 훈련하기

비율-왜곡 Lagrangian, 즉 비율과 왜곡의 합을 최적화하는 방식으로 트레이너를 컴파일합니다. 여기서 한 가지 항은 Lagrange 매개변수 $\lambda$에 의해 가중됩니다.

이 손실 함수는 모델의 다른 부분에 다르게 영향을 줍니다.

- 분석 변환은 비율과 왜곡 사이의 원하는 절충을 실현하는 잠재 표현을 생성하도록 훈련됩니다.
- 합성 변환은 잠재적 표현이 주어지면 왜곡을 최소화하도록 훈련됩니다.
- Prior의 매개변수는 잠재 표현이 주어지면 비율을 최소화하도록 훈련됩니다. 이는 최대 가능성의 의미에서 잠재 공간의 한계 분포로 prior를 피팅하는 것과 동일합니다.

In [ ]:
def pass_through_loss(_, x):
  # Since rate and distortion are unsupervised, the loss doesn't need a target.
  return x

def make_mnist_compression_trainer(lmbda, latent_dims=50):
  trainer = MNISTCompressionTrainer(latent_dims)
  trainer.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    # Just pass through rate and distortion as losses/metrics.
    loss=dict(rate=pass_through_loss, distortion=pass_through_loss),
    metrics=dict(rate=pass_through_loss, distortion=pass_through_loss),
    loss_weights=dict(rate=1., distortion=lmbda),
  )
  return trainer


다음으로 모델을 훈련시킵니다. 사람의 주석은 여기에서 필요하지 않습니다. 우리는 단지 이미지를 압축하기를 원하기 때문에 `map`을 사용하여 이미지를 삭제하고 대신 비율과 왜곡에 대해 "더미" 타겟을 추가합니다.

In [ ]:
def add_rd_targets(image, label):
  # Training is unsupervised, so labels aren't necessary here. However, we
  # need to add "dummy" targets for rate and distortion.
  return image, dict(rate=0., distortion=0.)

def train_mnist_model(lmbda):
  trainer = make_mnist_compression_trainer(lmbda)
  trainer.fit(
      training_dataset.map(add_rd_targets).batch(128).prefetch(8),
      epochs=15,
      validation_data=validation_dataset.map(add_rd_targets).batch(128).cache(),
      validation_freq=1,
      verbose=1,
  )
  return trainer

trainer = train_mnist_model(lmbda=2000)


## 일부 MNIST 이미지 압축하기

테스트 시간에 압축 및 압축 해제를 위해 훈련된 모델을 두 부분으로 나눕니다.

- 인코더 쪽은 분석 변환과 엔트로피 모델로 구성됩니다.
- 디코더 쪽은 합성 변환과 동일한 엔트로피 모델로 구성됩니다.

테스트 시간에 잠재 공간에는 추가 노이즈가 없지만 양자화되고 손실 없이 압축되므로 새로운 이름을 지정합니다. 우리는 이것과 이미지 재구성을 각각 $\hat x$ 및 $\hat y$라고 부릅니다([End-to-end Optimized Image Compression](https://arxiv.org/abs/1611.01704)을 따름).

In [ ]:
class MNISTCompressor(tf.keras.Model):
  """Compresses MNIST images to strings."""

  def __init__(self, analysis_transform, entropy_model):
    super().__init__()
    self.analysis_transform = analysis_transform
    self.entropy_model = entropy_model

  def call(self, x):
    # Ensure inputs are floats in the range (0, 1).
    x = tf.cast(x, self.compute_dtype) / 255.
    y = self.analysis_transform(x)
    # Also return the exact information content of each digit.
    _, bits = self.entropy_model(y, training=False)
    return self.entropy_model.compress(y), bits


In [ ]:
class MNISTDecompressor(tf.keras.Model):
  """Decompresses MNIST images from strings."""

  def __init__(self, entropy_model, synthesis_transform):
    super().__init__()
    self.entropy_model = entropy_model
    self.synthesis_transform = synthesis_transform

  def call(self, string):
    y_hat = self.entropy_model.decompress(string, ())
    x_hat = self.synthesis_transform(y_hat)
    # Scale and cast back to 8-bit integer.
    return tf.saturate_cast(tf.round(x_hat * 255.), tf.uint8)


`compression=True`로 인스턴스화하면 엔트로피 모델은 훈련된 prior를 범위 코딩 알고리즘에 대한 테이블로 변환합니다. `compress()`를 호출할 때 이 알고리즘이 호출되어 잠재 공간 벡터를 비트 시퀀스로 변환합니다. 각 바이너리 문자열의 길이는 잠재 공간의 내용에 근사합니다(prior 아래 잠재 공간의 음의 로그 가능성).

범위 코딩 테이블이 양쪽에서 정확히 동일해야 하기 때문에 압축 및 압축 해제를 위한 엔트로피 모델은 동일한 인스턴스여야 합니다. 그렇지 않으면 디코딩 오류가 발생할 수 있습니다.

In [ ]:
def make_mnist_codec(trainer, **kwargs):
  # The entropy model must be created with `compression=True` and the same
  # instance must be shared between compressor and decompressor.
  entropy_model = tfc.ContinuousBatchedEntropyModel(
      trainer.prior, coding_rank=1, compression=True, **kwargs)
  compressor = MNISTCompressor(trainer.analysis_transform, entropy_model)
  decompressor = MNISTDecompressor(entropy_model, trainer.synthesis_transform)
  return compressor, decompressor

compressor, decompressor = make_mnist_codec(trainer)


검증 데이터세트에서 16개의 이미지를 가져옵니다. 인수를 `skip`으로 변경하여 다른 하위 집합을 선택할 수 있습니다.

In [ ]:
(originals, _), = validation_dataset.batch(16).skip(3).take(1)


이를 문자열로 압축하고 각 내용을 비트 단위로 추적합니다.

In [ ]:
strings, entropies = compressor(originals)

print(f"String representation of first digit in hexadecimal: 0x{strings[0].numpy().hex()}")
print(f"Number of bits actually needed to represent it: {entropies[0]:0.2f}")


문자열에서 이미지를 다시 압축 해제합니다.

In [ ]:
reconstructions = decompressor(strings)


압축된 바이너리 표현 및 재구성된 숫자와 함께 16개의 원래 숫자를 각각 표시합니다.

In [ ]:
#@title

def display_digits(originals, strings, entropies, reconstructions):
  """Visualizes 16 digits together with their reconstructions."""
  fig, axes = plt.subplots(4, 4, sharex=True, sharey=True, figsize=(12.5, 5))
  axes = axes.ravel()
  for i in range(len(axes)):
    image = tf.concat([
        tf.squeeze(originals[i]),
        tf.zeros((28, 14), tf.uint8),
        tf.squeeze(reconstructions[i]),
    ], 1)
    axes[i].imshow(image)
    axes[i].text(
        .5, .5, f"→ 0x{strings[i].numpy().hex()} →\n{entropies[i]:0.2f} bits",
        ha="center", va="top", color="white", fontsize="small",
        transform=axes[i].transAxes)
    axes[i].axis("off")
  plt.subplots_adjust(wspace=0, hspace=0, left=0, right=1, bottom=0, top=1)


In [ ]:
display_digits(originals, strings, entropies, reconstructions)


인코딩된 문자열의 길이는 각 숫자의 내용과 다릅니다.

이는 범위 코딩 프로세스가 이산 확률로 작동하고 약간의 오버헤드가 있기 때문입니다. 따라서 특히 짧은 문자열의 경우 해당 일치는 대략적인 것입니다. 그러나 범위 코딩은 **점근적으로 최적**입니다. 한도 내에서 예상 비트 수는 교차 엔트로피(예상되는 내용)에 접근하며, 이에 대한 훈련 모델의 비율 항은 상한입니다.

## 비율-왜곡 절충

위의 모델은 각 숫자를 나타내는 데 사용된 평균 비트 수와 재구성 시 발생한 오차 사이의 특정 절충(`lmbda=2000`으로 지정)을 찾도록 훈련되었습니다.

다른 값으로 실험을 반복하면 어떻게 될까요?

우선 $\lambda$를 500으로 줄여보겠습니다.

In [ ]:
def train_and_visualize_model(lmbda):
  trainer = train_mnist_model(lmbda=lmbda)
  compressor, decompressor = make_mnist_codec(trainer)
  strings, entropies = compressor(originals)
  reconstructions = decompressor(strings)
  display_digits(originals, strings, entropies, reconstructions)

train_and_visualize_model(lmbda=500)


숫자의 충실도와 마찬가지로 코드의 비트 전송률이 낮아집니다. 그러나 대부분의 숫자는 여전히 인식할 수 있습니다.

$\lambda$를 더 줄여보겠습니다.

In [ ]:
train_and_visualize_model(lmbda=300)


문자열은 이제 숫자당 1바이트 정도로 훨씬 짧아지기 시작합니다. 그러나 여기에는 대가가 따릅니다. 더 많은 숫자가 인식할 수 없게 됩니다.

이것은 이 모델이 오류에 대한 인간의 인식을 고려하지 않으며 픽셀 값의 관점에서 절대 편차만 측정한다는 것을 보여줍니다. 더 잘 인식되는 이미지 품질을 얻으려면 픽셀 손실을 인지 손실로 대체해야 합니다.

## 디코더를 생성 모델로 사용하기

디코더에 임의의 비트를 공급하면 모델이 숫자를 나타내도록 학습한 분포에서 효과적으로 샘플링됩니다.

먼저, 입력 문자열이 완전히 디코딩되지 않았는지 감지하는 온전성 검사 없이 압축기/압축 해제기를 다시 인스턴스화합니다.

In [ ]:
compressor, decompressor = make_mnist_codec(trainer, decode_sanity_check=False)


이제 충분히 긴 임의의 문자열을 압축 해제기에 공급하여 숫자를 디코딩/샘플링할 수 있도록 합니다.

In [ ]:
import os

strings = tf.constant([os.urandom(8) for _ in range(16)])
samples = decompressor(strings)

fig, axes = plt.subplots(4, 4, sharex=True, sharey=True, figsize=(5, 5))
axes = axes.ravel()
for i in range(len(axes)):
  axes[i].imshow(tf.squeeze(samples[i]))
  axes[i].axis("off")
plt.subplots_adjust(wspace=0, hspace=0, left=0, right=1, bottom=0, top=1)
